In [20]:
import numpy as np
import pandas as pd
from functools import reduce, partial
# import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
%pip install torch
# faiss is not supported on macOS with CUDA, so you can remove it if it's not needed.
# import faiss
# from faiss.contrib.ondisk import merge_ondisk
import os, re, glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from collections import namedtuple
import pickle as pkl
from datetime import timezone, datetime, timedelta
import pytz
# from sentence_transformers import SentenceTransformer, util

from tqdm.auto import tqdm
from tqdm.contrib import tenumerate

# Replaced cuml imports (CUDA-based) with sklearn and other alternatives
from sklearn.ensemble import RandomForestClassifier as rfc  # CPU-based Random Forest
from sklearn.svm import SVC  # CPU-based Support Vector Machine
from sklearn.linear_model import LogisticRegression  # CPU-based Logistic Regression

from sklearn.metrics import accuracy_score  # Replaced cuml.metrics with sklearn.metrics for CPU support

from xgboost import XGBClassifier  # xgboost is CPU-based by default, you don't need a GPU version

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


Add empty folders 'processed', 'embeddings', 'index'

Put all pre-processed .csv files in the 'processed' folder. the files need to have a column named 'sentence'.

Upload models-2023-04-10-2229.pkl and trained.index
run the code below to install relevant dependencies.

In [22]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)
def files(dir):
  return sorted_alphanumeric([dir + x for x in os.listdir(dir)])

In [23]:
model = SentenceTransformer('msmarco-distilbert-base-v4') # asymmetric

taxonomy = {
    "competence": {
        "competence_trust": ["intelligent", "reasoned", "informed", "evidence", "accurate", "truthful"],
        "competence_distrust": ["stupid", "incompetent", "ignorant", "idiot", "sheep", "insane", "moron",
                                "dumbass", "clown", "living in your bubble", "incoherent", "nonsense", "no proof",
                                "accept reality", "irrational", "retarded", "intellectually dishonest",
                                "misleading", "indoctrinated"] # new
    },
    "sincerity": {
        "sincerity_trust": ["trustworthy", "reputable", "sincere", "genuine"],
        "sincerity_distrust": ["liar", "dishonest", "untrustworthy", "corrupt", "inhuman", "immoral",
                               "disinformation", "misinformation", "propaganda", "fake news", "unreliable source", "paranoid", "shill",
                               "bias", "discredited", "manipulated", "scam"] # new
    }
}

In [24]:
def flatten_taxonomy(taxonomy, keys=[]):
  if type(taxonomy) is not dict:
    return [(x, keys) for x in taxonomy]
  return reduce(lambda x, y: x + y, [flatten_taxonomy(t, keys + [k]) for k, t in taxonomy.items()])

In [25]:
def t2i_i2t_embeddings(flat_taxonomy, model):
  t2i = {k: i for i, (k, v) in enumerate(flat_taxonomy)}
  i2t = {i: k for i, (k, v) in enumerate(flat_taxonomy)}
  embs = model.encode([i2t[i] for i in range(len(flat_taxonomy))], normalize_embeddings=True)
  return (t2i, i2t, embs)

In [26]:
flat_taxonomy = flatten_taxonomy(taxonomy)
t2i, i2t, taxonomy_embs = t2i_i2t_embeddings(flat_taxonomy, model)
taxonomy_embs.shape

(46, 768)

flat_taxonomy -->

[('intelligent', ['competence', 'competence_trust']),

 ('reasoned', ['competence', 'competence_trust']),

 ('informed', ['competence', 'competence_trust']),..]

t2i -->

{'intelligent': 0,

 'reasoned': 1,

 'informed': 2,

 'evidence': 3,...}

In [27]:
## You don't need the below code -- I provide a trained index file. But here it is for reference!
## Embed a sample of sentences and train index
# embeddings = model.encode(sentences_sample, batch_size=128, normalize_embeddings=True)

# index = faiss.index_factory(768, "OPQ64_256,IVF262144_HNSW32,PQ64") # train on embeddings...

# index_ivf = faiss.extract_index_ivf(index)
# clustering_index = faiss.index_cpu_to_all_gpus(faiss.IndexFlatL2(index_ivf.d))
# index_ivf.clustering_index = clustering_index

# index.train(embeddings) # training on GPU -- 42 mins vs > 5hrs!!
# index_ivf.clustering_index = faiss.index_gpu_to_cpu(index_ivf.clustering_index)
# faiss.write_index(index, "trained.index")

In [28]:
csvs = files("processed/")

In [29]:
import os
import pandas as pd
import numpy as np
import pickle as pkl
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'taxonomy_embs' is already loaded or defined
taxonomy_embs = ...  # Load or define your taxonomy embeddings here

offset_total = 0
csvs = [...]  # List of CSV files to process

# Initialize SentenceTransformer model (if not already initialized)
model = SentenceTransformer('all-MiniLM-L6-v2')

for i, csv in enumerate(csvs):
    if '.ipynb' not in csv:
        print(f"Processing {csv}")
        
        # Read CSV file
        df = pd.read_csv(csv)
        sents = [str(x) for x in df["sentence"]]
        
        # Encode the sentences to embeddings
        embs = model.encode(sents, normalize_embeddings=True)
        
        # Compute cosine similarity between the taxonomy and sentence embeddings
        sims = cosine_similarity(taxonomy_embs, embs)
        
        # Save the similarities to a pickle file
        with open(f"embeddings/embs-{os.path.basename(csv)}.pkl", "wb") as f:
            pkl.dump(sims, f)

        # Simulating a local storage of embeddings (instead of using faiss index)
        # For simplicity, we will use numpy arrays to store the embeddings
        
        print(f"Writing embeddings for block_{os.path.basename(csv)}.npy...")
        
        # Save the embeddings to a file (using NumPy)
        np.save(f"embeddings/block_{os.path.basename(csv)}.npy", embs)

        # Simulate a simple index by just saving the embeddings
        print(f"Index for block_{os.path.basename(csv)}.index...")
        with open(f"index/block_{os.path.basename(csv)}.index", "wb") as f:
            pkl.dump(embs, f)
        
        # Update the offset for next batch of embeddings
        offset_total += len(df)

TypeError: argument of type 'ellipsis' is not iterable

In [ ]:
## Running models

def construct_data(prompts, sims, t2i, threshold=0.5, neg_to_pos_ratio=1):
  sent_emb_idx_positive = set()
  sent_emb_idx_negative = set()
  #sims = util.cos_sim(model.encode(prompts, normalize_embeddings=True), sentence_embeddings)
  sims = sims[[t2i[x] for x in prompts]] # keeps row logic working below
  for i, prompt in enumerate(prompts):
    #sorted_idxs = torch.argsort(-sims[i])
    pos_idxs = torch.argwhere(sims[i] >= threshold).flatten().tolist()
    neg_idxs = torch.argwhere(sims[i] < threshold).flatten().tolist()
    pos = set(pos_idxs)
    neg = set(np.random.choice(neg_idxs, round(len(pos) * neg_to_pos_ratio)))
    sent_emb_idx_positive = sent_emb_idx_positive.union(pos)
    sent_emb_idx_negative = sent_emb_idx_negative.union(neg)
  return {"positive": list(sent_emb_idx_positive), "negative": list(sent_emb_idx_negative)}

In [ ]:
def expand_data(data, df, model, index, nprobe=2048, k=3, expand=True):
  if not expand:
    return []
  query_embs = model.encode(df.iloc[data["positive"]]["sentence"].tolist(), normalize_embeddings=True) #embeddings[data["positive"]]
  index.nprobe = nprobe
  D, I = index.search(query_embs, k)
  new_sents = I.flatten().tolist()
  print(f"Expanded {query_embs.shape[0]} examples by {len(new_sents)} to {query_embs.shape[0] + len(new_sents)}")
  return new_sents

In [ ]:
# Use the pretrained index...
index = faiss.read_index("trained.index")
blocks = files("index/")
merge_ondisk(index, blocks, "merged_index.ivfdata")
faiss.write_index(index, "populated.index")
#index.ntotal

In [ ]:
# Construct and expand data
eptrust_categories = ["competence_distrust", "competence_trust", "sincerity_distrust", "sincerity_trust"]
mdl_data = {category: {"pos": [], "neg": [], "pos_sims": [], "neg_sims": [], "pos_nn": []} for category in eptrust_categories}

csvs = files("processed/")

for csv in tqdm(csvs):
  print(f"##### Processing {csv}...")
  df = pd.read_csv(csv)
  with open(f"embeddings/embs-{os.path.basename(csv)}.pkl", "rb") as f:
    sims = pkl.load(f)

  for category in eptrust_categories:
    print(f"# Processing {category}...")
    qry = [x for x, y in flat_taxonomy if category in y]
    data = construct_data(qry, sims, t2i, neg_to_pos_ratio=20) # df.iloc[x["positive"]]
    pos_expand = expand_data(data, df, model, index, nprobe=2048, k=5, expand=True)

    mdl_data[category]["pos"].append(df.iloc[data["positive"]])
    mdl_data[category]["neg"].append(df.iloc[data["negative"]])
    mdl_data[category]["pos_sims"].append(sims[:, data["positive"]])
    mdl_data[category]["neg_sims"].append(sims[:, data["negative"]])
    mdl_data[category]["pos_nn"].append(pos_expand)


mdl_nn_data = {category: {"pos_nn_index": {}, "pos_nn_data": []} for category in eptrust_categories}
n_per_file = 100000

for category in eptrust_categories:
  pos_nn = set(reduce(lambda x, y: x + y, mdl_data[category]["pos_nn"]))
  pos_nn_index = {}

  for x in pos_nn:
    if (x // n_per_file) not in pos_nn_index:
      pos_nn_index[x // n_per_file] = []
    pos_nn_index[x // n_per_file].append(x % n_per_file)
  mdl_nn_data[category]["pos_nn_index"] = pos_nn_index
#print(pos_nn_index)
for file_id, csv in tenumerate(csvs):
  for category in eptrust_categories:
    df = pd.read_csv(csv)
    if file_id not in pos_nn_index:
      print(f"Nothing in {csv}... weird.")
    else:
      pos_nn_index = mdl_nn_data[category]["pos_nn_index"]
      #print(pos_nn_index)
      #print(file_id)
      #### NOTE: this is probably due to trained.index using 1000000-sized chunks and our.index files using 100000-sized chunks.
      #### Contact Dominic for details on trianing our own train.index/getting more RAM space to use larger chunk sizes.
      valid_indices = [i for i in pos_nn_index[file_id] if 0 <= i < len(df)]
      if valid_indices:
        mdl_nn_data[category]["pos_nn_data"].append(df.iloc[valid_indices])
      else:
        print(f"No valid indices found for file {file_id} in category {category}")

In [ ]:
# Now to collate the data and train models
def prepare_data(category, mdl_data, mdl_nn_data, model, taxonomy_embs, i2t, raw=False):
  print(f"Preparing {category}...")
  pos_mat = torch.concat(mdl_data[category]["pos_sims"], dim=1)
  pos_data = pd.DataFrame(pos_mat.numpy()).rename(i2t).transpose()

  neg_mat = torch.concat(mdl_data[category]["neg_sims"], dim=1)
  neg_data = pd.DataFrame(neg_mat.numpy()).rename(i2t).transpose()

  df_expanded = pd.concat(mdl_nn_data[category]["pos_nn_data"])
  pos_nn_embs = model.encode(df_expanded["sentence"].tolist(), normalize_embeddings=True)
  pos_nn_mat = util.cos_sim(taxonomy_embs, pos_nn_embs)
  pos_nn_data = pd.DataFrame(pos_nn_mat.numpy()).rename(i2t).transpose()
  print(f"Positive: {pos_mat.shape}, Negative: {neg_mat.shape}, Expanded: {pos_nn_mat.shape}")
  pos_data = pd.concat([pos_data, pos_nn_data])

  if raw:
    pos_mat = model.encode(pd.concat(mdl_data[category]["pos"])["sentence"].tolist(), normalize_embeddings=True)
    neg_mat = model.encode(pd.concat(mdl_data[category]["neg"])["sentence"].tolist(), normalize_embeddings=True)
    pos_data = pd.concat([pd.DataFrame(pos_mat), pd.DataFrame(pos_nn_embs)])
    neg_data = pd.DataFrame(neg_mat)
    print(f"Positive: {pos_mat.shape}, Negative: {neg_mat.shape}, Expanded: {pos_nn_embs.shape}")

  pos_data["label"] = 1
  neg_data["label"] = 0
  data = pd.concat([pos_data, neg_data], ignore_index=True)

  return data

In [ ]:
def run_mdl(data, rf_max_depth=5, test=0.3):
  y_data = data["label"]
  x_data = data.drop("label", axis = 1)
  x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x_data, y_data, test_size = test)

  #lm = LogisticRegression(penalty="none", solver="saga", max_iter=1000), MLPClassifier(random_state=1, hidden_layer_sizes=(384, 192, 96))
  mdl = curfc(max_depth=rf_max_depth) # this one standard
  #mdl = MLPClassifier(random_state=1, hidden_layer_sizes=(24, 12, 6))
  #mdl = XGBClassifier(n_estimators=4, max_depth=rf_max_depth, learning_rate=1, objective='binary:logistic')
  mdl.fit(x_training_data, y_training_data)
  predictions = mdl.predict(x_test_data)
  print(classification_report(y_test_data, predictions))
  return mdl

In [ ]:
def run_mdl_full(data, estimator):
  y_data = data["label"]
  x_data = data.drop("label", axis = 1)

  mdl = estimator(None)
  mdl.fit(x_data, y_data)
  return mdl

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

estimators = {
    "RF": lambda x: RandomForestClassifier(max_depth=10, random_state=1),  # Replaced curfc with sklearn's RandomForestClassifier
    "MLP": lambda x: MLPClassifier(random_state=1, hidden_layer_sizes=(24, 12, 6)),
    "XGB": lambda x: XGBClassifier(n_estimators=4, max_depth=10, learning_rate=1, objective='binary:logistic'),
    "LR": lambda x: LogisticRegression(),
    "SVM": lambda x: SVC(kernel='rbf', degree=3, gamma='auto', C=1)  # Uncommented and included the SVM model
}

In [ ]:
def write_sentence_data(mdl_data, mdl_nn_data, category, outfile):
  pos = pd.concat(mdl_data[category]["pos"] + mdl_nn_data[category]["pos_nn_data"])
  neg = pd.concat(mdl_data[category]["neg"])
  pos["label"] = 1
  neg["label"] = 0
  print(f"Positive: {pos.shape}, Negative: {neg.shape}, outfile: {outfile}")
  pd.concat([pos, neg]).to_csv(outfile, index=False)

In [ ]:
write_sentence_data(mdl_data, mdl_nn_data, "competence_distrust", "competence_distrust_data.csv")
write_sentence_data(mdl_data, mdl_nn_data, "sincerity_distrust", "sincerity_distrust_data.csv")

In [ ]:
# Finally!
prepped_data = {category: prepare_data(category, mdl_data, mdl_nn_data, model, taxonomy_embs, i2t, raw=False) for category in eptrust_categories}
mdls = {
    est_name: {
        cat: run_mdl_full(df, estimator) for cat, df in prepped_data.items()
    } for est_name, estimator in estimators.items()
}

In [ ]:
def cls_(sents, mdls, model, taxonomy_embs, i2t, raw=False, sims=None):
  if sims is None:
    embs = model.encode(sents, normalize_embeddings=True)
    test_data = pd.DataFrame(util.cos_sim(taxonomy_embs, embs).numpy()).rename(i2t).transpose()
  else:
    test_data = pd.DataFrame(sims.numpy()).rename(i2t).transpose()
  if raw:
    test_data = pd.DataFrame(embs)
  df_res = pd.concat([mdl.predict_proba(test_data)[1].rename(cat) for cat, mdl in mdls.items()], axis=1) # RFs
  #df_res = pd.concat([pd.Series(mdl.predict_proba(test_data)[:,1]).rename(cat) for cat, mdl in mdls.items()], axis=1) # almost everything else lol
  return df_res.rename(index={i: s for i, s in enumerate(sents)})

In [ ]:
cls = partial(cls_, mdls=mdls["RF"], model=model, taxonomy_embs=taxonomy_embs, i2t=i2t)

In [ ]:
# Example table in the paper
sents = ["you are ignorant and a liar", "democrats are idiots", "democrats are liars", "republicans are dumb", "you're smart", "democrats are intelligent, trustworthy", "Alice is trustworthy, but Bob is stupid."]
res = cls(sents)
res.round(4)

threshold = 0.5
res.mask(res > threshold, "*").mask(res <= threshold, "")

In [ ]:
# Readout models
fname = "models-" + datetime.today().astimezone(pytz.timezone('US/Central')).strftime('%Y-%m-%d-%H%M') + ".pkl"
with open(fname, "wb") as f:
  pkl.dump(mdls, f)

In [ ]:
# Running the models on your own data, assuming it's a dataframe with a "sentence" column (feel free to precompute with the "sims" arg):
cls(sents=df["sentence"].tolist())

In [ ]:
import pickle
with open('/content/models-2023-04-10-2229.pkl', 'rb') as file:
    d_mdls = pickle.load(file)

In [ ]:
df_before = pd.read_csv('/content/processed/before_april_chunck_0.csv')
df_after = pd.read_csv('/content/processed/after_april_chunck_0.csv')

In [ ]:
!pip install --upgrade treelite

In [ ]:
cls2 = partial(cls_, mdls=d_mdls["RF"], model=model, taxonomy_embs=taxonomy_embs, i2t=i2t)
result_before = cls2(sents=df_before["sentence"].tolist())
result_before

In [ ]:
sents = ["you are ignorant and a liar", "democrats are idiots", "democrats are liars", "republicans are dumb", "you're smart", "democrats are intelligent, trustworthy", "Alice is trustworthy, but Bob is stupid."]
res = cls2(sents)
res.round(4)

threshold = 0.5
res.mask(res > threshold, "*").mask(res <= threshold, "")